In [13]:
import numpy as np
import tensorflow.compat.v2 as tf
import tensorflow_probability as tfp
tfb = tfp.bijectors
tfd = tfp.distributions
tfk = tfp.math.psd_kernels
tf.enable_v2_behavior()

In [14]:
# Compute the RMSE given the ground truth (y_true) and the predictions(y_pred)
def root_mean_squared_error(y_true, y_pred):
        return tf.math.sqrt(tf.math.reduce_mean(tf.math.square(y_pred - y_true), axis=-1))
    
class InputTransformedKernel(tfp.math.psd_kernels.PositiveSemidefiniteKernel):

    def __init__(self, kernel, transformation, name='InputTransformedKernel'):
        self._kernel = kernel
        self._transformation = transformation
        super(InputTransformedKernel, self).__init__(
            feature_ndims=kernel.feature_ndims,
            dtype=kernel.dtype,
            name=name)

    def apply(self, x1, x2):
        return self._kernel.apply(
            self._transformation(x1),
            self._transformation(x2))

    def matrix(self, x1, x2):
        return self._kernel.matrix(
            self._transformation(x1),
            self._transformation(x2))

    @property
    def batch_shape(self):
        return self._kernel.batch_shape

    def batch_shape_tensor(self):
        return self._kernel.batch_shape_tensor

class InputScaledKernel(InputTransformedKernel):

    def __init__(self, kernel, length_scales):
        super(InputScaledKernel, self).__init__(
            kernel,
            lambda x: x / tf.expand_dims(length_scales,
                                     -(kernel.feature_ndims + 1)))        

In [15]:
# Load labeled data
data = np.loadtxt('../data/labeled_data.dat')
x_labeled = data[:, :2].astype(np.float64) # -2 because we do not need porosity predictions
y_labeled = data[:, -2:-1].astype(np.float64) # dimensionless bond length and porosity measurements

# Normalize the data.
from sklearn import preprocessing

# normalize dataset with MinMaxScaler
scaler = preprocessing.MinMaxScaler(feature_range=(0.0, 1.0))
x_labeled = scaler.fit_transform(x_labeled)
# y_labeled = scaler.fit_transform(y_labeled)

tr_size = 30

# train and test data
trainX, trainY = x_labeled[:tr_size,:], y_labeled[:tr_size]
testX, testY = x_labeled[tr_size:,:], y_labeled[tr_size:]

trainY = np.transpose(trainY)
testY = np.transpose(testY)

data_phyloss = np.loadtxt('../data/unlabeled_data_BK_constw_v2_1525.dat')
x_unlabeled = data_phyloss[:, :]

# initial porosity
initporo = x_unlabeled[:, -1]

x_unlabeled1 = x_unlabeled[:1303, :2]
x_unlabeled2 = x_unlabeled[-6:, :2]
x_unlabeled = np.vstack((x_unlabeled1,x_unlabeled2))

x_unlabeled = scaler.fit_transform(x_unlabeled)
init_poro1 = initporo[:1303]
init_poro2 = initporo[-6:]
init_poro = np.hstack((init_poro1,init_poro2))

In [16]:
def build_gp(amplitude, length_scale):
    """Defines the conditional dist. of GP outputs, given kernel parameters."""

    # Create the covariance kernel, which will be shared between the prior (which we
    # use for maximum likelihood training) and the posterior (which we use for
    # posterior predictive sampling)    
    se_kernel = tfk.ExponentiatedQuadratic(amplitude)  # length_scale = None here, implicitly

    # This is the "ARD" kernel (we don't like abbreviations or bizarrely obscure names in
    # TFP, so we're probably going to call this "InputScaledKernel" since....that's what it is! :)
    kernel = InputScaledKernel(se_kernel, length_scale)
    
    # Create the GP prior distribution, which we will use to train the model
    # parameters.
    return tfd.GaussianProcess(kernel=kernel,index_points=trainX)

gp_joint_model = tfd.JointDistributionNamedAutoBatched({
    'amplitude': tfd.TransformedDistribution(
            distribution=tfd.Normal(loc=0., scale=np.float64(1.)),
            bijector=tfb.Exp(),
            batch_shape=[1]),
    'length_scale': tfd.TransformedDistribution(
            distribution=tfd.Normal(loc=0., scale=np.float64(1.)),
            bijector=tfb.Exp(),
            batch_shape=[2]),
    'observations': build_gp,
})

In [17]:
# Create the trainable model parameters, which we'll subsequently optimize.
# Note that we constrain them to be strictly positive.
constrain_positive = tfb.Shift(np.finfo(np.float64).tiny)(tfb.Exp())

amplitude_var = tfp.util.TransformedVariable(
    initial_value=np.random.uniform(size=1),
    bijector=constrain_positive,
    name='amplitude',
    dtype=np.float64)

length_scale_var = tfp.util.TransformedVariable(
    initial_value=np.random.uniform(size=[2]),
    bijector=constrain_positive,
    name='length_scale',
    dtype=np.float64)

trainable_variables = [v.trainable_variables[0] for v in 
                       [amplitude_var,
                       length_scale_var]]

In [18]:
@tf.function(autograph=False, experimental_compile=False)
def target_log_prob(amplitude, length_scale, poroi, lam):
    tf.random.set_seed(1234)
    se_kernel = tfk.ExponentiatedQuadratic(amplitude)  # length_scale = None here, implicitly
    optimized_kernel = InputScaledKernel(se_kernel, length_scale)
    gprm = tfd.GaussianProcessRegressionModel(kernel=optimized_kernel, index_points = x_unlabeled)
    samples = gprm.sample(1)
    pred = tf.squeeze(samples, axis=0)

    phyloss_poro = tf.math.reduce_mean(tf.nn.relu(tf.negative(pred))+tf.nn.relu(pred-poroi))

#     print("phyloss_poro:",lam*phyloss_poro)
#     return lam*phyloss_poro
    return lam*phyloss_poro - gp_joint_model.log_prob({
      'amplitude': amplitude,
      'length_scale': length_scale,
      'observations': trainY
  })

In [19]:
tf.random.set_seed(1234)

# Optimize the model parameters.
num_iters = 40
lam = 100000
optimizer = tf.optimizers.Adam(learning_rate=.1)

# Store the likelihood values during training, so we can plot the progress
lls_ = np.zeros(num_iters, np.float64)

for i in range(num_iters):
    with tf.GradientTape() as tape:
        loss = target_log_prob(amplitude_var, length_scale_var, init_poro, lam) # physics loss & normal loss


    print(i,"loss_inloop:",loss)
    grads = tape.gradient(loss, trainable_variables)
    optimizer.apply_gradients(zip(grads, trainable_variables))
    lls_[i] = loss

print('Trained parameters:')
print('amplitude: {}'.format(amplitude_var._value().numpy()))
print('length_scale: {}'.format(length_scale_var._value().numpy()))

0 loss_inloop: tf.Tensor(97327.17451122966, shape=(), dtype=float64)
1 loss_inloop: tf.Tensor(27517.653673109533, shape=(), dtype=float64)
2 loss_inloop: tf.Tensor(26125.867318869172, shape=(), dtype=float64)
3 loss_inloop: tf.Tensor(16993.676754054417, shape=(), dtype=float64)
4 loss_inloop: tf.Tensor(63578.97302255117, shape=(), dtype=float64)
5 loss_inloop: tf.Tensor(3150.835495812226, shape=(), dtype=float64)
6 loss_inloop: tf.Tensor(23689.326876749325, shape=(), dtype=float64)
7 loss_inloop: tf.Tensor(26075.86432005829, shape=(), dtype=float64)
8 loss_inloop: tf.Tensor(33741.71708333892, shape=(), dtype=float64)
9 loss_inloop: tf.Tensor(35355.67059762244, shape=(), dtype=float64)
10 loss_inloop: tf.Tensor(17614.334487859152, shape=(), dtype=float64)
11 loss_inloop: tf.Tensor(43646.772000166406, shape=(), dtype=float64)
12 loss_inloop: tf.Tensor(14456.26112218793, shape=(), dtype=float64)
13 loss_inloop: tf.Tensor(19869.55961840706, shape=(), dtype=float64)
14 loss_inloop: tf.Tenso

In [20]:
import numpy as np
import tensorflow.compat.v2 as tf
import tensorflow_probability as tfp
tfb = tfp.bijectors
tfd = tfp.distributions
tfk = tfp.math.psd_kernels
tf.enable_v2_behavior()


import random

def pass_arg(Xx, nsim, tr_size):
    print("Tr_size:", tr_size)
    def fix_seeds(seed):
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
        sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
    #     K.set_session(sess)
        tf.compat.v1.keras.backend.set_session(sess)

    ss = 1
    fix_seeds(ss)

    # Compute the RMSE given the ground truth (y_true) and the predictions(y_pred)
    def root_mean_squared_error(y_true, y_pred):
            return tf.math.sqrt(tf.math.reduce_mean(tf.math.square(y_pred - y_true), axis=-1))

    class InputTransformedKernel(tfp.math.psd_kernels.PositiveSemidefiniteKernel):

        def __init__(self, kernel, transformation, name='InputTransformedKernel'):
            self._kernel = kernel
            self._transformation = transformation
            super(InputTransformedKernel, self).__init__(
                feature_ndims=kernel.feature_ndims,
                dtype=kernel.dtype,
                name=name)

        def apply(self, x1, x2):
            return self._kernel.apply(
                self._transformation(x1),
                self._transformation(x2))

        def matrix(self, x1, x2):
            return self._kernel.matrix(
                self._transformation(x1),
                self._transformation(x2))

        @property
        def batch_shape(self):
            return self._kernel.batch_shape

        def batch_shape_tensor(self):
            return self._kernel.batch_shape_tensor

    class InputScaledKernel(InputTransformedKernel):

        def __init__(self, kernel, length_scales):
            super(InputScaledKernel, self).__init__(
                kernel,
                lambda x: x / tf.expand_dims(length_scales,
                                         -(kernel.feature_ndims + 1)))

    # Load labeled data
    data = np.loadtxt('../data/labeled_data.dat')
    x_labeled = data[:, :2].astype(np.float64) # -2 because we do not need porosity predictions
    y_labeled = data[:, -2:-1].astype(np.float64) # dimensionless bond length and porosity measurements

    # Normalize the data.
    from sklearn import preprocessing

    # normalize dataset with MinMaxScaler
    scaler = preprocessing.MinMaxScaler(feature_range=(0.0, 1.0))
    x_labeled = scaler.fit_transform(x_labeled)
    # y_labeled = scaler.fit_transform(y_labeled)

    tr_size = 30

    # train and test data
    trainX, trainY = x_labeled[:tr_size,:], y_labeled[:tr_size]
    testX, testY = x_labeled[tr_size:,:], y_labeled[tr_size:]

    trainY = np.transpose(trainY)
    testY = np.transpose(testY)

    data_phyloss = np.loadtxt('../data/unlabeled_data_BK_constw_v2_1525.dat')
    x_unlabeled = data_phyloss[:, :]

    # initial porosity
    initporo = x_unlabeled[:, -1]

    x_unlabeled1 = x_unlabeled[:1303, :2]
    x_unlabeled2 = x_unlabeled[-6:, :2]
    x_unlabeled = np.vstack((x_unlabeled1,x_unlabeled2))

    x_unlabeled = scaler.fit_transform(x_unlabeled)
    init_poro1 = initporo[:1303]
    init_poro2 = initporo[-6:]
    init_poro = np.hstack((init_poro1,init_poro2))


    def build_gp(amplitude, length_scale):
        """Defines the conditional dist. of GP outputs, given kernel parameters."""

        # Create the covariance kernel, which will be shared between the prior (which we
        # use for maximum likelihood training) and the posterior (which we use for
        # posterior predictive sampling)    
        se_kernel = tfk.ExponentiatedQuadratic(amplitude)  # length_scale = None here, implicitly

        # This is the "ARD" kernel (we don't like abbreviations or bizarrely obscure names in
        # TFP, so we're probably going to call this "InputScaledKernel" since....that's what it is! :)
        kernel = InputScaledKernel(se_kernel, length_scale)

        # Create the GP prior distribution, which we will use to train the model
        # parameters.
        return tfd.GaussianProcess(kernel=kernel,index_points=trainX)

    gp_joint_model = tfd.JointDistributionNamedAutoBatched({
        'amplitude': tfd.TransformedDistribution(
                distribution=tfd.Normal(loc=0., scale=np.float64(1.)),
                bijector=tfb.Exp(),
                batch_shape=[1]),
        'length_scale': tfd.TransformedDistribution(
                distribution=tfd.Normal(loc=0., scale=np.float64(1.)),
                bijector=tfb.Exp(),
                batch_shape=[2]),
        'observations': build_gp,
    })					



    # Create the trainable model parameters, which we'll subsequently optimize.
    # Note that we constrain them to be strictly positive.
    constrain_positive = tfb.Shift(np.finfo(np.float64).tiny)(tfb.Exp())

    amplitude_var = tfp.util.TransformedVariable(
        initial_value=np.random.uniform(size=1),
        bijector=constrain_positive,
        name='amplitude',
        dtype=np.float64)

    length_scale_var = tfp.util.TransformedVariable(
        initial_value=np.random.uniform(size=[2]),
        bijector=constrain_positive,
        name='length_scale',
        dtype=np.float64)

    trainable_variables = [v.trainable_variables[0] for v in 
                           [amplitude_var,
                           length_scale_var]]



    @tf.function(autograph=False, experimental_compile=False)
    def target_log_prob(amplitude, length_scale, poroi, lam):
        tf.random.set_seed(1234)
        se_kernel = tfk.ExponentiatedQuadratic(amplitude)  # length_scale = None here, implicitly
        optimized_kernel = InputScaledKernel(se_kernel, length_scale)
        gprm = tfd.GaussianProcessRegressionModel(kernel=optimized_kernel, index_points = x_unlabeled)
        samples = gprm.sample(1)
        pred = tf.squeeze(samples, axis=0)

        phyloss_poro = tf.math.reduce_mean(tf.nn.relu(tf.negative(pred))+tf.nn.relu(pred-poroi))

    #     print("phyloss_poro:",lam*phyloss_poro)
    #     return lam*phyloss_poro
        return lam*phyloss_poro - gp_joint_model.log_prob({
          'amplitude': amplitude,
          'length_scale': length_scale,
          'observations': trainY
      })


    tf.random.set_seed(1234)

    # Optimize the model parameters.
    num_iters = 40
    lam = 100000
    optimizer = tf.optimizers.Adam(learning_rate=.1)

    # Store the likelihood values during training, so we can plot the progress
    lls_ = np.zeros(num_iters, np.float64)

    for i in range(num_iters):
        with tf.GradientTape() as tape:
            loss = target_log_prob(amplitude_var, length_scale_var, init_poro, lam) # physics loss & normal loss


        # print(i,"loss_inloop:",loss)
        grads = tape.gradient(loss, trainable_variables)
        optimizer.apply_gradients(zip(grads, trainable_variables))
        lls_[i] = loss

    # print('Trained parameters:')
    # print('amplitude: {}'.format(amplitude_var._value().numpy()))
    # print('length_scale: {}'.format(length_scale_var._value().numpy()))



    tf.random.set_seed(1234)
    se_kernel = tfk.ExponentiatedQuadratic(amplitude_var)  # length_scale = None here, implicitly
    optimized_kernel = InputScaledKernel(se_kernel, length_scale_var)
    gprm = tfd.GaussianProcessRegressionModel(kernel=optimized_kernel, index_points = testX)
    samples = gprm.sample(1)
    samples

In [28]:
tf.random.set_seed(1234)
se_kernel = tfk.ExponentiatedQuadratic(amplitude_var)  # length_scale = None here, implicitly
optimized_kernel = InputScaledKernel(se_kernel, length_scale_var)
gprm = tfd.GaussianProcessRegressionModel(kernel=optimized_kernel, index_points = testX)
samples = gprm.sample(2)
# optimized_kernel = tfk.ExponentiatedQuadratic(amplitude_var, length_scale_var)
# gpr = tfd.GaussianProcessRegressionModel(kernel=optimized_kernel, index_points = testX)
# samples = gpr.sample(1)
samples

<tf.Tensor: shape=(2, 1, 9), dtype=float64, numpy=
array([[[-0.09655712, -0.09761954, -0.06867765,  0.08486226,
          0.07861124,  0.11342095,  0.09207502, -0.07753923,
          0.03311804]],

       [[ 0.04212455,  0.03606457,  0.06086621,  0.0506872 ,
         -0.11496251, -0.09564325, -0.02192549,  0.0386103 ,
          0.02477364]]])>

In [30]:
np.array(tf.squeeze(samples, axis=1)).shape

(2, 9)

In [22]:
testY

array([[0.0105933 , 0.03315482, 0.01352198, 0.05269549, 0.00490424,
        0.00895544, 0.01003564, 0.01345683, 0.02496436]])

In [23]:
root_mean_squared_error(testY, samples)

<tf.Tensor: shape=(1, 1), dtype=float64, numpy=array([[0.08679325]])>

In [24]:
import matplotlib.pyplot as plt
# Plot the loss evolution
plt.figure(figsize=(12, 4))
plt.plot(lls_)
plt.xlabel("Training iteration")
plt.ylabel("Log marginal likelihood")
plt.show()

ModuleNotFoundError: No module named 'matplotlib'